In [61]:
import pandas as pd
import re


df = pd.read_csv('animes_wh.csv', engine='python', on_bad_lines='skip')


# Check
print(df.head())
print(df.columns)

     uid                             title  \
0  28891           Haikyuu!! Second Season   
1  23273           Shigatsu wa Kimi no Uso   
2  34599                     Made in Abyss   
3   5114  Fullmetal Alchemist: Brotherhood   
4  31758  Kizumonogatari III: Reiketsu-hen   

                                            synopsis  \
0  Following their participation at the Inter-Hig...   
1  Music accompanies the path of the human metron...   
2  The Abyss—a gaping chasm stretching down into ...   
3  "In order for something to be obtained, someth...   
4  After helping revive the legendary vampire Kis...   

                                               genre  \
0  ['Comedy', 'Sports', 'Drama', 'School', 'Shoun...   
1  ['Drama', 'Music', 'Romance', 'School', 'Shoun...   
2  ['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...   
3  ['Action', 'Military', 'Adventure', 'Comedy', ...   
4   ['Action', 'Mystery', 'Supernatural', 'Vampire']   

                          aired  episodes  member

In [62]:
pip install requests beautifulsoup4 textblob

In [63]:
import requests
from bs4 import BeautifulSoup

def fetch_description(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad responses
        soup = BeautifulSoup(response.text, 'html.parser')

        # Adjust the selector based on the structure of the webpage
        description_element = soup.find('div', class_='synopsis')  # Change this as needed

        if description_element:
            return description_element.get_text(strip=True)
        else:
            print(f"No description found for URL: {url}")
            return "Description not available."
    except Exception as e:
        print(f"Description: {url}: {e}")
        return ""

In [64]:
df['description'] = df['synopsis'].apply(fetch_description)

# Check
print(df[['title', 'synopsis']].head(10))

Streaming output truncated to the last 5000 lines.
In addition, an ancient secret society that threatens the peace of both humans and Mazoku has risen, and a fight for the throne that now leaves the nobles wondering who is the legitimate Maou. ?
Description: When five students at Yamaboshi Academy realize that there are no clubs where they fit in, they band together to form the Student Cultural Society, or "StuCS" for short. The club consists of: Taichi Yaegashi, a hardcore wrestling fan; Iori Nagase, an indecisive optimist; Himeko Inaba, a calm computer genius; Yui Kiriyama, a petite karate practitioner; and Yoshifumi Aoki, the class clown. 
  
One day, Aoki and Yui experience a strange incident when, without warning, they switch bodies for a short period of time. As this supernatural phenomenon continues to occur randomly amongst the five friends, they begin to realize that it is not just fun and games. Now forced to become closer than ever, they soon discover each other's hidden sec

In [65]:
from textblob import TextBlob
def clean_text(text):
    if isinstance(text, str):
        # Remove HTML tags
        text = re.sub(r'<.*?>', '', text)
        return text.strip()
    return ""  # Return an empty string for non-string inputs

# Clean
df['cleaned_synopsis'] = df['synopsis'].apply(clean_text)

# Function to get sentiment polarity(-1,0,1)
def get_sentiment(text):
    # Ensure the text is a string
    if not isinstance(text, str):
        text = str(text)  # Convert to string if it's not
    return TextBlob(text).sentiment.polarity  # Returns a value between -1 and 1

# Apply sentiment analysis
df['sentiment'] = df['cleaned_synopsis'].apply(get_sentiment)

# Check sentiment
print(df[['title', 'sentiment']].head())

                              title  sentiment
0           Haikyuu!! Second Season   0.176475
1           Shigatsu wa Kimi no Uso   0.223214
2                     Made in Abyss   0.152447
3  Fullmetal Alchemist: Brotherhood  -0.021627
4  Kizumonogatari III: Reiketsu-hen   0.189374


In [71]:
def mood_to_sentiment(mood):
    mood_mapping = {
        'happy': (0.1, 1.0),
        'sad': (-1.0, -0.1),
        'excited': (0.5, 1.0),
        'relaxed': (0.0, 0.5),
        'angry': (-1.0, -0.5),
        'nostalgic': (0.0, 0.5),
        'adventurous': (0.3, 1.0),
        'romantic': (0.2, 1.0),
        'melancholic': (-0.5, 0.0),
        'motivated': (0.3, 1.0),
        'curious': (0.0, 1.0),
        'hopeful': (0.2, 1.0),
        'bored': (-0.5, 0.0),
        'fearful': (-1.0, -0.3),
        'confused': (-0.5, 0.0),
        'pensive': (0.0, 0.5),
        'playful': (0.3, 1.0),
        'determined': (0.3, 1.0),
        'surprised': (0.0, 1.0),
        'skeptical': (-0.5, 0.0),
        'grateful': (0.2, 1.0),
        'lonely': (-1.0, -0.1),
        'content': (0.0, 0.5),
        'anxious': (-1.0, -0.3),
        'empowered': (0.3, 1.0),
        'silly': (0.3, 1.0),
        'reflective': (0.0, 0.5),
        'inspired': (0.3, 1.0),
        'disappointed': (-0.5, 0.0),
        'peaceful': (0.0, 0.5),
        'confident': (0.1, 0.8),
        'sexy': (0.0, 0.5),
        'energetic': (0.3, 1.0),
        'nonchalant': (-0.8, 0.0),
        'suicidal': (0.5, 1.0),
        'shocked': (0.0, 0.5),
        'passionate': (0.3, 1.0),
        'peaceful': (0.0, 0.5),
        'nothing': (-1, 1),
        'optimistic': (0.3, 1.0),
        'gloomy': (-0.8, -0.2),
        'grumpy': (-0.6, -0.1),
        'elated': (0.7, 1.0),
        'tense': (-0.7, -0.2),
        'embarrassed': (-0.5, 0.0),
        'jealous': (-0.6, -0.1),
        'blissful': (0.5, 1.0),
        'apathetic': (-0.5, 0.0),
        'awed': (0.4, 1.0),
        'vindicated': (0.4, 1.0),
        'resentful': (-0.6, -0.2),
        'ecstatic': (0.8, 1.0),
        'frustrated': (-0.7, -0.2),
        'ashamed': (-0.6, -0.1)
    }
    return mood_mapping.get(mood, None)

In [72]:
from IPython.display import display, HTML

def recommend_anime(mood):
    sentiment_range = mood_to_sentiment(mood)
    output = ""  # Initialize output variable
    if sentiment_range:
        lower, upper = sentiment_range
        print(f"Filtering for sentiment range: {lower} to {upper}")

        # Filter based on sentiment
        recommendations = df[(df['sentiment'] >= lower) & (df['sentiment'] <= upper)]
        print(f"Number of entries after sentiment filtering: {len(recommendations)}")

        if recommendations.empty:
            return "No recommendations found for the given mood."

        # Prepare HTML output for displaying images
        for index, row in recommendations.iterrows():
            output += f"""
            <div style="margin-bottom: 20px;">
                <img src="{row['img_url']}" alt="{row['title']}" width="200"/>
                <h3>{row['title']}</h3>
                <p><strong>Genre:</strong> {row['genre']}</p>
                <p><strong>Episodes:</strong> {row['episodes']}</p>
                <p><strong>Rating:</strong> {row['score']}</p>
                <p><strong>Ranking:</strong> {row['ranked']}</p>
                <p><strong>Aired:</strong> {row['aired']}</p>
                <p><strong>Description:</strong> {row['synopsis']}</p>
            </div>
            """

        # Display the recommendations with images
        display(HTML(output))
    else:
        return "Mood not recognized."

In [ ]:
user_mood = 'happy'  # This would come from your UI input
recommended_animes = recommend_anime(user_mood)

if isinstance(recommended_animes, str):
    print(recommended_animes)
else:
    print("Recommended Animes:")
    print(recommended_animes)

Filtering for sentiment range: -0.6 to -0.1
Number of entries after sentiment filtering: 393


Recommended Animes:
None
